<a href="https://colab.research.google.com/github/Dipeshpal/Advance-Text-Topics-in-Tensorflow-2.0/blob/main/zero_shot_classification_GPT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from transformers import AutoTokenizer, AutoModel
from torch.nn import functional as F
import json

tokenizer = AutoTokenizer.from_pretrained('deepset/sentence_bert')
model = AutoModel.from_pretrained('deepset/sentence_bert')


Some weights of the model checkpoint at deepset/sentence_bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [33]:
file_path = "/content/sample_data/laptop.json"

f = open(file_path,)
data = json.load(f)
  
name_li = []
for i in data:
  name_li.append(i['product_name'])
  
# Closing file
f.close()

sentence = name_li[:20]
labels = ['64 GB', '128 GB', '256 GB', '512 GB', '1 TB']
print(sentence)

[['Asus R465JA Core i3-1005G1 4GB 128GB 14 Inch Full HD Windows 10 S Laptop - R465JA-EK058T'], ['Dell Latitude 3510 Core i5-10210U 8GB 256GB SSD 15.6 Inch Windows 10 Pro Laptop - VCFVM'], ['Lenovo V15-IIL Core i5-1035G1 8GB 512GB SSD 15.6 Inch Full HD Windows 10 Laptop - 82C500G4UK'], ['Lenovo ThinkPad E15 Core i7-10510U 16GB 512GB SSD 15.6 Inch FHD Windows 10 Pro Laptop - 20RD0011UK'], ['HP 250 G7 Core i5-1035G1 8GB 256GB SSD 15.6 Inch Windows 10 Pro Laptop - 14Z88EA'], ['HP 250 G7 Core i5-1035G1 8GB 256GB SSD 15.6 Inch FHD Windows 10 Home Laptop - 15L03ES'], ['Lenovo V14-ADA Athlon Gold 3150U 8GB 256GB SSD 14 Inch Full HD Windows 10 Home Laptop - 82C6005CUK'], ['Lenovo V15-ADA AMD Ryzen 5-3500U 8GB 256GB SSD 15.6 Inch FHD Windows 10 Pro Laptop - 82C70006UK'], ['Lenovo V15 Althlon Silver 3050U 4GB 128GB SSD 15.6 Inch FHD Windows 10 Laptop - 82C700E4UK'], ['Asus C523 Intel Celeron N3350 4GB 64GB eMMC 15.6 Inch Chromebook - C523NA-BR0067'], ['Asus VivoBook R429MA-BV286TS Celeron N4000 4

In [37]:
def get_output(sent, labl):
  sentence = sent
  labels = labl

  # run inputs through model and mean-pool over the sequence
  # dimension to get sequence-level representations
  inputs = tokenizer.batch_encode_plus([sentence] + labels,
                                      return_tensors='pt',
                                      pad_to_max_length=True)
  input_ids = inputs['input_ids']
  attention_mask = inputs['attention_mask']
  output = model(input_ids, attention_mask=attention_mask)[0]
  sentence_rep = output[:1].mean(dim=1)
  label_reps = output[1:].mean(dim=1)

  # now find the labels with the highest cosine similarities to
  # the sentence
  similarities = F.cosine_similarity(sentence_rep, label_reps)
  closest = similarities.argsort(descending=True)
  for ind in closest:
      print(f'label: {labels[ind]} \t similarity: {similarities[ind]}')

In [39]:
for i in sentence:
  print(i[0])
  get_output(i[0], ['32 GB', '64 GB', '128GB', '256GB', '512GB', '1TB'])
  print("-------------")

Asus R465JA Core i3-1005G1 4GB 128GB 14 Inch Full HD Windows 10 S Laptop - R465JA-EK058T


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


label: 128GB 	 similarity: 0.5040740966796875
label: 64 GB 	 similarity: 0.4159872233867645
label: 256GB 	 similarity: 0.38052648305892944
label: 512GB 	 similarity: 0.31228166818618774
label: 32 GB 	 similarity: 0.2987075746059418
label: 1TB 	 similarity: 0.22793258726596832
-------------
Dell Latitude 3510 Core i5-10210U 8GB 256GB SSD 15.6 Inch Windows 10 Pro Laptop - VCFVM
label: 256GB 	 similarity: 0.39091265201568604
label: 64 GB 	 similarity: 0.3374136984348297
label: 512GB 	 similarity: 0.3313816487789154
label: 128GB 	 similarity: 0.2930392622947693
label: 32 GB 	 similarity: 0.27225372195243835
label: 1TB 	 similarity: 0.06114767864346504
-------------
Lenovo V15-IIL Core i5-1035G1 8GB 512GB SSD 15.6 Inch Full HD Windows 10 Laptop - 82C500G4UK
label: 512GB 	 similarity: 0.45660126209259033
label: 128GB 	 similarity: 0.4063299298286438
label: 256GB 	 similarity: 0.35200175642967224
label: 64 GB 	 similarity: 0.3311329782009125
label: 32 GB 	 similarity: 0.28968051075935364
labe